<a href="https://colab.research.google.com/github/edsondamasceno/classification-COVID-19/blob/main/Genetic_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier as mlp
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import cohen_kappa_score as kappa
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
def populacao_inicial(tamanho_populacao, tamanho_cromossomo):
    return np.random.choice([0, 1], size=(tamanho_populacao,tamanho_cromossomo))

In [ ]:
# One-point crossover
def crossover_two_best_n_random(populacao_atual,fitness_populacao,tamanho_cromossomo,n = 5):

    tamanho_populacao = len(populacao_atual)
    filhos = np.zeros((tamanho_populacao,tamanho_cromossomo))
    cont_filhos = 0
    for i in range(tamanho_populacao//2):

        tamanho_populacao_crossover = len(populacao_atual)#esse tamanho muda a cada iteracao

        if tamanho_populacao_crossover >= n: #escolhe os 5 somente se houverem pelo menos 5 na lista
            escolhidos = np.random.choice(range(tamanho_populacao_crossover),size = n)#escolhe n pais aleatorios
        two_best = np.argsort(fitness_populacao)[-2:] #escolhe os 2 melhores entre os n
        pai = populacao_atual[two_best[0]]
        mae = populacao_atual[two_best[1]]

        populacao_atual = np.delete(populacao_atual,(two_best), axis=0) #remove os pais da populacao atual
        fitness_populacao = np.delete(fitness_populacao,(two_best)) #remove os fitness para ficar igual a lista de pais

        point_cross = np.random.choice(range(1,tamanho_cromossomo-1))
        filho1 = np.concatenate((pai[:point_cross],mae[point_cross:]))
        filho2 = np.concatenate((mae[:point_cross],pai[point_cross:]))

        filhos[cont_filhos,:] = filho1
        filhos[cont_filhos+1,:] = filho2
        cont_filhos += 2

    return filhos

In [ ]:
# bitwise mutation
def mutation(filhos, percentage=5):
    for id_f,f in enumerate(filhos):
        prob = np.random.choice(range(100))
        if prob > (100-percentage):
            #print('X-men')
            position = np.random.choice(range(0,tamanho_cromossomo))
            f[position] = int(not(f[position])) #troca 0->1 ou 1->0
            filhos[id_f] = f
    return filhos

In [ ]:
def fitness(individuo):

    ## selecionar as features de acordo com o cromossomo: 1-> feature entra
    col = np.where(individuo==1)
    selected_features_train = features_train[:,col[0]]
    selected_features_validation = features_validation[:,col[0]]

    ## classificacao
    clf = rf()
    clf.fit(selected_features_train,labels_train)
    pred = clf.predict(selected_features_validation)
    k = kappa(labels_validation,pred)

    return k

In [ ]:
def next_generation(populacao_atual,fitness_populacao,filhos,fitness_filhos,tamanho_populacao):

    populacao = np.concatenate((populacao_atual,filhos))
    fitness = np.concatenate((fitness_populacao,fitness_filhos))
    next_generation = np.argsort(fitness)[-tamanho_populacao:] #escolhe os 'tamanho_populacao' melhores individuos
    populacao = populacao[next_generation]
    fitness = fitness[next_generation]
    return populacao,fitness

In [ ]:
def condicao_parada(fitness_populacao):
    tamanho_populacao = len(fitness_populacao)

    if (np.sum(fitness_populacao) == tamanho_populacao): #condicao para saber se todos os individuos possuem 100% de acerto
        print('Parou com 100% de acerto')
        return False
    return True

## Ler CSV

In [ ]:
file = '/content/drive/MyDrive/Paper CBM/Feature/Feature_Otimizada.csv'
dados = np.loadtxt(file,delimiter=',',skiprows=1)
labels = dados[:,-1]
dados = dados[:,:-1]
# dividir dados: 70% para treino e validacao (onde 80% treino e 20% validacao) e 30% para teste
features_evolutivo, features_test, labels_evolutivo, labels_test = train_test_split(dados, labels, test_size=0.10)
features_train, features_validation, labels_train, labels_validation = train_test_split(features_evolutivo, labels_evolutivo, test_size=0.20)

## Parâmetros do sistema evolutivo

In [ ]:
tamanho_populacao = 50
tamanho_cromossomo = dados.shape[1]
quantidade_geracoes = 100
taxa_mutacao = 5

## Montando Evolutivo

In [ ]:
populacao_atual = populacao_inicial(tamanho_populacao,tamanho_cromossomo)
fitness_populacao = [fitness(i) for i in populacao_atual]
print(fitness_populacao)

In [ ]:
qg = 0
while ((qg != quantidade_geracoes)): #or (condicao_parada(fitness_populacao)):
    filhos = crossover_two_best_n_random(populacao_atual,fitness_populacao,tamanho_cromossomo)
    filhos = mutation(filhos)
    fitness_filhos = [fitness(i) for i in filhos]
    populacao_atual,fitness_populacao = next_generation(populacao_atual,fitness_populacao,filhos,fitness_filhos,tamanho_populacao)
    print(qg)
    qg += 1

## Buscar menor quantidade de Features

In [ ]:
quant_best_individuos = np.sum(fitness_populacao == fitness_populacao.max())
print(quant_best_individuos)
ordenado = np.argsort(fitness_populacao)[-quant_best_individuos:] #pega o indice do(s) melhor(es)
if quant_best_individuos > 1:
    #pegar o individuo com menos atributos
    tam_best = tamanho_cromossomo
    for i in ordenado:
        if np.sum(populacao_atual[i]) <= tam_best:
            best = i
            tam_best = np.sum(populacao_atual[i])
individuo_final = populacao_atual[best]

## Ajustar vetores para selecionar os atributos

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def evaluate(individuo):

    ## selecionar as features de acordo com o cromossomo: 1-> feature entra
    col = np.where(individuo==1)
    selected_features_evolutivo = features_evolutivo[:,col[0]]
    selected_features_test = features_test[:,col[0]]

    ## classificacao
    #clf = mlp()
    clf = rf()
    clf.fit(selected_features_evolutivo,labels_evolutivo)
    pred = clf.predict(selected_features_test)
    k = kappa(labels_test,pred)
    recall = recall_score(labels_test, pred, average='macro')
    precision = precision_score(labels_test, pred, average='macro')
    auc_score = roc_auc_score(labels_test, pred, average="macro")
    f1 = f1_score(labels_test, pred, average='macro')
    accuracy = accuracy_score(labels_test, pred)

    print("========================== Result =========================")
    print("Accuracy:", (accuracy*100))
    print("Recall:", (recall*100))
    print("Precision:", (precision*100))
    print("F1-score:", f1)
    print("AUROC:", auc_score)
    print("Kappa:", k)
    print("===========================================================")

    #return k

In [ ]:
evaluate(individuo_final)